In [1]:
# 기본 라이브러리 임포트
import pandas as pd             
import numpy as np             
import matplotlib.pyplot as plt 
import seaborn as sns           
import gc                       # 가비지 컬렉션(메모리 해제)
import re                       # 정규 표현식 처리
from collections import defaultdict  # 기본값이 있는 딕셔너리 생성

# 경고 메시지 억제
import warnings
warnings.filterwarnings('ignore')   

# 그래프 스타일 설정
sns.set()                           # seaborn 기본 스타일 적용

# matplotlib 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # 한글 폰트 설정
# plt.rcParams['font.family'] = 'AppleGothic'  
plt.rcParams['figure.figsize'] = (12, 6)       # 그림 크기 설정 (가로, 세로)
plt.rcParams['font.size'] = 14                 # 폰트 크기 설정
plt.rcParams['axes.unicode_minus'] = False     # 마이너스 기호 깨짐 방지

# 결측치 시각화 라이브러리 임포트
import missingno                          # 결측치 분포를 시각화하는 유틸리티

# 범주형 변수 레이블 인코딩을 위한 도구 임포트
from sklearn.preprocessing import LabelEncoder

# 중복 조합 생성에 사용할 product 함수 임포트
from itertools import product

# 회귀 및 통계 분석을 위한 statsmodels 임포트
import statsmodels.api as sm

# 다중공선성 진단용 VIF 계산 함수 임포트
from statsmodels.stats.outliers_influence import variance_inflation_factor

### 데이터 병합

In [2]:
# 채널정보 파일 읽기
channel_df = pd.read_parquet('open/concat/2018_채널정보.parquet')

# 회원정보 파일 읽기
member_df  = pd.read_parquet('open/concat/2018_회원정보.parquet')

# 회원정보에서 ID와 Segment 컬럼만 추출
member_seg = member_df[['ID', 'Segment']]

# 중복된 ID 개수 확인
dup_count = member_seg['ID'].duplicated().sum()
print(f'중복된 ID 개수: {dup_count}')

# 중복된 ID를 첫 번째 항목만 남기고 제거
member_seg_unique = member_seg.drop_duplicates(subset='ID', keep='first')
print(f'중복 제거 후 행 수: {len(member_seg_unique)}')

# 채널정보에 Segment 컬럼 병합 (1:1 조인 보장)
df = channel_df.merge(member_seg_unique, on='ID', how='left')

# 결과 확인
print("병합 후 데이터프레임 크기:", df.shape)
print(df.head())

# 병합된 파일 저장
df.to_parquet('2018_채널정보_with_segment.parquet', index=False)

중복된 ID 개수: 2500000
중복 제거 후 행 수: 500000
병합 후 데이터프레임 크기: (3000000, 106)
     기준년월            ID 인입횟수_ARS_R6M 이용메뉴건수_ARS_R6M  인입일수_ARS_R6M  \
0  201807  TRAIN_000000       10회 이상         10회 이상             8   
1  201807  TRAIN_000001        1회 이상          1회 이상             0   
2  201807  TRAIN_000002        1회 이상          1회 이상             1   
3  201807  TRAIN_000003       10회 이상         10회 이상            10   
4  201807  TRAIN_000004        1회 이상          1회 이상             0   

   인입월수_ARS_R6M  인입후경과월_ARS  인입횟수_ARS_B0M  이용메뉴건수_ARS_B0M  인입일수_ARS_B0M  ...  \
0             6           0             2               6             2  ...   
1             0           0             0               0             0  ...   
2             1           0             2               5             1  ...   
3             6           0             2               6             2  ...   
4             0           0             0               0             0  ...   

  당사PAY_방문월수_R6M 당사멤버쉽_방문횟수_B0M  당

In [3]:
# 데이터를 불러온다.
df = pd.read_parquet('2018_채널정보_with_segment.parquet')
df

,기준년월,ID,인입횟수_ARS_R6M,이용메뉴건수_ARS_R6M,인입일수_ARS_R6M,인입월수_ARS_R6M,인입후경과월_ARS,인입횟수_ARS_B0M,이용메뉴건수_ARS_B0M,인입일수_ARS_B0M,...,당사PAY_방문월수_R6M,당사멤버쉽_방문횟수_B0M,당사멤버쉽_방문횟수_R6M,당사멤버쉽_방문월수_R6M,OS구분코드,홈페이지_금융건수_R6M,홈페이지_선결제건수_R6M,홈페이지_금융건수_R3M,홈페이지_선결제건수_R3M,Segment
0,201807,TRAIN_000000,10회 이상,10회 이상,8,6,0,2,6,2,...,0,22,221,6,Android,0,0,0,0,D
1,201807,TRAIN_000001,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,None,0,0,0,0,E
2,201807,TRAIN_000002,1회 이상,1회 이상,1,1,0,2,5,1,...,0,0,0,0,Android,11,6,5,5,C
3,201807,TRAIN_000003,10회 이상,10회 이상,10,6,0,2,6,2,...,0,23,219,6,Android,0,0,0,0,D
4,201807,TRAIN_000004,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,Android,0,0,0,0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,201812,TEST_99995,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,None,0,0,0,0,None
2999996,201812,TEST_99996,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,None,3,7,3,3,None
2999997,201812,TEST_99997,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,None,0,0,0,0,None
2999998,201812,TEST_99998,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,Android,7,22,4,12,None


In [4]:
# 기준년월 값의 하위 두 자리가 12인(12월) 행만 선택
df = df[df['기준년월'] % 100 == 12].copy()
df

,기준년월,ID,인입횟수_ARS_R6M,이용메뉴건수_ARS_R6M,인입일수_ARS_R6M,인입월수_ARS_R6M,인입후경과월_ARS,인입횟수_ARS_B0M,이용메뉴건수_ARS_B0M,인입일수_ARS_B0M,...,당사PAY_방문월수_R6M,당사멤버쉽_방문횟수_B0M,당사멤버쉽_방문횟수_R6M,당사멤버쉽_방문월수_R6M,OS구분코드,홈페이지_금융건수_R6M,홈페이지_선결제건수_R6M,홈페이지_금융건수_R3M,홈페이지_선결제건수_R3M,Segment
2000000,201812,TRAIN_000000,10회 이상,30회 이상,12,6,0,2,6,2,...,0,23,136,6,Android,0,0,0,0,D
2000001,201812,TRAIN_000001,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,None,0,0,0,0,E
2000002,201812,TRAIN_000002,1회 이상,1회 이상,1,1,5,0,0,0,...,0,0,0,0,Android,12,10,6,6,C
2000003,201812,TRAIN_000003,10회 이상,30회 이상,12,6,0,2,6,2,...,0,45,219,6,Android,0,0,0,0,D
2000004,201812,TRAIN_000004,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,Android,0,0,0,0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,201812,TEST_99995,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,None,0,0,0,0,None
2999996,201812,TEST_99996,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,None,3,7,3,3,None
2999997,201812,TEST_99997,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,None,0,0,0,0,None
2999998,201812,TEST_99998,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,Android,7,22,4,12,None


In [5]:
# 기준년월 컬럼 제거
df.drop(columns=['기준년월'], inplace=True)
df

,ID,인입횟수_ARS_R6M,이용메뉴건수_ARS_R6M,인입일수_ARS_R6M,인입월수_ARS_R6M,인입후경과월_ARS,인입횟수_ARS_B0M,이용메뉴건수_ARS_B0M,인입일수_ARS_B0M,방문횟수_PC_R6M,...,당사PAY_방문월수_R6M,당사멤버쉽_방문횟수_B0M,당사멤버쉽_방문횟수_R6M,당사멤버쉽_방문월수_R6M,OS구분코드,홈페이지_금융건수_R6M,홈페이지_선결제건수_R6M,홈페이지_금융건수_R3M,홈페이지_선결제건수_R3M,Segment
2000000,TRAIN_000000,10회 이상,30회 이상,12,6,0,2,6,2,1회 이상,...,0,23,136,6,Android,0,0,0,0,D
2000001,TRAIN_000001,1회 이상,1회 이상,0,0,0,0,0,0,1회 이상,...,0,0,0,0,None,0,0,0,0,E
2000002,TRAIN_000002,1회 이상,1회 이상,1,1,5,0,0,0,20회 이상,...,0,0,0,0,Android,12,10,6,6,C
2000003,TRAIN_000003,10회 이상,30회 이상,12,6,0,2,6,2,1회 이상,...,0,45,219,6,Android,0,0,0,0,D
2000004,TRAIN_000004,1회 이상,1회 이상,0,0,0,0,0,0,1회 이상,...,0,0,0,0,Android,0,0,0,0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,TEST_99995,1회 이상,1회 이상,0,0,0,0,0,0,1회 이상,...,0,0,0,0,None,0,0,0,0,None
2999996,TEST_99996,1회 이상,1회 이상,0,0,0,0,0,0,1회 이상,...,0,0,0,0,None,3,7,3,3,None
2999997,TEST_99997,1회 이상,1회 이상,0,0,0,0,0,0,1회 이상,...,0,0,0,0,None,0,0,0,0,None
2999998,TEST_99998,1회 이상,1회 이상,0,0,0,0,0,0,1회 이상,...,0,0,0,0,Android,7,22,4,12,None


### 결측치 찾기

In [6]:
# 전체 행 개수 계산
total_rows = len(df)

# 각 컬럼의 결측치 개수 계산
missing_count = df.isna().sum()

# 결측치 비율 계산 (전체 행 대비 %)
missing_pct = (missing_count / total_rows * 100).round(2)

# 결측치 정보 데이터프레임으로 정리
missing_info = pd.DataFrame({
    'missing_count': missing_count,
    'missing_pct': missing_pct
})

# 결측치가 있는 컬럼만 출력
print("\n결측치 정보 (개수 및 전체 대비 비율 %):")
print(missing_info[missing_info['missing_count'] > 0])


결측치 정보 (개수 및 전체 대비 비율 %):
         missing_count  missing_pct
OS구분코드          340203        68.04
Segment         100000        20.00


### 값이 하나 뿐인 컬럼과 값이 모두 0인 컬럼도 제외

In [7]:
# 전부 0인 컬럼 찾기
zero_cols = [c for c in df.columns if df[c].eq(0).all()]

print("모두 0인 컬럼 (제거 대상):")
print(zero_cols)

# 전부 0인 컬럼 제거
df.drop(columns=zero_cols, inplace = True)

# 4) 결과 확인
print("최종 DataFrame shape:", df.shape)
print("남은 컬럼:", df.columns.tolist())

모두 0인 컬럼 (제거 대상):
['인입횟수_금융_IB_R6M', '인입불만횟수_IB_R6M', '인입불만일수_IB_R6M', '인입불만월수_IB_R6M', '인입불만횟수_IB_B0M', '인입불만일수_IB_B0M', 'IB문의건수_사용승인내역_B0M', 'IB문의건수_한도_B0M', 'IB문의건수_결제_B0M', 'IB문의건수_할부_B0M', 'IB문의건수_정보변경_B0M', 'IB문의건수_결제일변경_B0M', 'IB문의건수_명세서_B0M', 'IB문의건수_비밀번호_B0M', 'IB문의건수_SMS_B0M', 'IB문의건수_APP_B0M', 'IB문의건수_부대서비스_B0M', 'IB문의건수_포인트_B0M', 'IB문의건수_카드발급_B0M', 'IB문의건수_BL_B0M', 'IB문의건수_분실도난_B0M', 'IB문의건수_CA_B0M', 'IB문의건수_CS_B0M', 'IB상담건수_VOC_B0M', 'IB상담건수_VOC민원_B0M', 'IB상담건수_VOC불만_B0M', 'IB상담건수_금감원_B0M', 'IB문의건수_한도_R6M', 'IB문의건수_결제_R6M', 'IB문의건수_할부_R6M', 'IB문의건수_정보변경_R6M', 'IB문의건수_결제일변경_R6M', 'IB문의건수_명세서_R6M', 'IB문의건수_비밀번호_R6M', 'IB문의건수_SMS_R6M', 'IB문의건수_APP_R6M', 'IB문의건수_부대서비스_R6M', 'IB문의건수_포인트_R6M', 'IB문의건수_BL_R6M', 'IB문의건수_분실도난_R6M', 'IB문의건수_CA_R6M', 'IB상담건수_VOC_R6M', 'IB상담건수_VOC민원_R6M', 'IB상담건수_VOC불만_R6M', 'IB상담건수_금감원_R6M', '불만제기건수_B0M', '불만제기건수_R12M', '당사PAY_방문횟수_B0M', '당사PAY_방문횟수_R6M', '당사PAY_방문월수_R6M']
최종 DataFrame shape: (500000, 55)
남은 컬럼: ['ID', '인입횟수_ARS_R6M', '이용메뉴건수_ARS_R6M

In [8]:
# 모든 값이 동일한(상수) 컬럼 찾기
const_cols = [c for c in df.columns if df[c].nunique(dropna=False) == 1]

# 제거 대상 컬럼 목록 출력
print("모두 같은 값인 컬럼 (제거 대상):")
print(const_cols)

# 상수 컬럼 제거
df.drop(columns=const_cols, inplace=True)

# 결과 확인
print("최종 DataFrame shape:", df.shape)
print("남은 컬럼:", df.columns.tolist())

모두 같은 값인 컬럼 (제거 대상):
['인입불만후경과월_IB_R6M']
최종 DataFrame shape: (500000, 54)
남은 컬럼: ['ID', '인입횟수_ARS_R6M', '이용메뉴건수_ARS_R6M', '인입일수_ARS_R6M', '인입월수_ARS_R6M', '인입후경과월_ARS', '인입횟수_ARS_B0M', '이용메뉴건수_ARS_B0M', '인입일수_ARS_B0M', '방문횟수_PC_R6M', '방문일수_PC_R6M', '방문월수_PC_R6M', '방문후경과월_PC_R6M', '방문횟수_앱_R6M', '방문일수_앱_R6M', '방문월수_앱_R6M', '방문후경과월_앱_R6M', '방문횟수_모바일웹_R6M', '방문일수_모바일웹_R6M', '방문월수_모바일웹_R6M', '방문후경과월_모바일웹_R6M', '방문횟수_PC_B0M', '방문일수_PC_B0M', '방문횟수_앱_B0M', '방문일수_앱_B0M', '방문횟수_모바일웹_B0M', '방문일수_모바일웹_B0M', '인입횟수_IB_R6M', '인입일수_IB_R6M', '인입월수_IB_R6M', '이용메뉴건수_IB_R6M', '인입후경과월_IB_R6M', '인입횟수_IB_B0M', '인입일수_IB_B0M', '이용메뉴건수_IB_B0M', '상담건수_B0M', 'IB문의건수_선결제_B0M', 'IB문의건수_CL_RV_B0M', '상담건수_R6M', 'IB문의건수_사용승인내역_R6M', 'IB문의건수_선결제_R6M', 'IB문의건수_카드발급_R6M', 'IB문의건수_CL_RV_R6M', 'IB문의건수_CS_R6M', '불만제기후경과월_R12M', '당사멤버쉽_방문횟수_B0M', '당사멤버쉽_방문횟수_R6M', '당사멤버쉽_방문월수_R6M', 'OS구분코드', '홈페이지_금융건수_R6M', '홈페이지_선결제건수_R6M', '홈페이지_금융건수_R3M', '홈페이지_선결제건수_R3M', 'Segment']


In [9]:
# 전체 컬럼 리스트 생성
cols = df.columns.tolist()

# 최빈값 비율이 99.9% 초과하는 컬럼 수집
low_variance_cols = []
for col in cols:
    # 계산: 컬럼별 최빈값 비율 확인
    top_ratio = df[col].value_counts(normalize=True, dropna=False).iloc[0]
    if top_ratio > 0.999:
        low_variance_cols.append(col)

# 제거 대상 컬럼 목록 출력
print("제거 대상 컬럼:", low_variance_cols)

# 제거 대상 컬럼 삭제
df.drop(columns=low_variance_cols, inplace=True)

# 결과 확인
print("최종 컬럼 리스트:", df.columns.tolist())

제거 대상 컬럼: ['방문횟수_모바일웹_R6M', '방문일수_모바일웹_R6M', '방문횟수_모바일웹_B0M', '방문일수_모바일웹_B0M', 'IB문의건수_CL_RV_B0M', 'IB문의건수_사용승인내역_R6M', 'IB문의건수_카드발급_R6M', 'IB문의건수_CL_RV_R6M', 'IB문의건수_CS_R6M']
최종 컬럼 리스트: ['ID', '인입횟수_ARS_R6M', '이용메뉴건수_ARS_R6M', '인입일수_ARS_R6M', '인입월수_ARS_R6M', '인입후경과월_ARS', '인입횟수_ARS_B0M', '이용메뉴건수_ARS_B0M', '인입일수_ARS_B0M', '방문횟수_PC_R6M', '방문일수_PC_R6M', '방문월수_PC_R6M', '방문후경과월_PC_R6M', '방문횟수_앱_R6M', '방문일수_앱_R6M', '방문월수_앱_R6M', '방문후경과월_앱_R6M', '방문월수_모바일웹_R6M', '방문후경과월_모바일웹_R6M', '방문횟수_PC_B0M', '방문일수_PC_B0M', '방문횟수_앱_B0M', '방문일수_앱_B0M', '인입횟수_IB_R6M', '인입일수_IB_R6M', '인입월수_IB_R6M', '이용메뉴건수_IB_R6M', '인입후경과월_IB_R6M', '인입횟수_IB_B0M', '인입일수_IB_B0M', '이용메뉴건수_IB_B0M', '상담건수_B0M', 'IB문의건수_선결제_B0M', '상담건수_R6M', 'IB문의건수_선결제_R6M', '불만제기후경과월_R12M', '당사멤버쉽_방문횟수_B0M', '당사멤버쉽_방문횟수_R6M', '당사멤버쉽_방문월수_R6M', 'OS구분코드', '홈페이지_금융건수_R6M', '홈페이지_선결제건수_R6M', '홈페이지_금융건수_R3M', '홈페이지_선결제건수_R3M', 'Segment']


### 비수치형 또는 혼합형 컬럼 찾아 숫자로 변환하기

In [10]:
# 각 컬럼별 분포(value_counts) 출력
c1 = list (df.columns)
for col in c1:
    print(f"▶ [{col}] 분포")
    print(df[col].value_counts(dropna=False))
    print()  # 빈 줄로 구분

▶ [ID] 분포
ID
TRAIN_000000    1
TRAIN_333343    1
TRAIN_333341    1
TRAIN_333340    1
TRAIN_333339    1
               ..
TRAIN_166662    1
TRAIN_166661    1
TRAIN_166660    1
TRAIN_166659    1
TEST_99999      1
Name: count, Length: 500000, dtype: int64

▶ [인입횟수_ARS_R6M] 분포
인입횟수_ARS_R6M
1회 이상     482727
10회 이상     17273
Name: count, dtype: int64

▶ [이용메뉴건수_ARS_R6M] 분포
이용메뉴건수_ARS_R6M
1회 이상     469779
30회 이상     12826
10회 이상      9907
20회 이상      7488
Name: count, dtype: int64

▶ [인입일수_ARS_R6M] 분포
인입일수_ARS_R6M
0     418648
1      44956
12     12484
2       8468
10      3975
4       2565
3       2520
8       2282
6       1885
5        928
7        569
9        431
11       289
Name: count, dtype: int64

▶ [인입월수_ARS_R6M] 분포
인입월수_ARS_R6M
0    400147
1     56348
2     25142
6     17428
3       689
4       172
5        74
Name: count, dtype: int64

▶ [인입후경과월_ARS] 분포
인입후경과월_ARS
0    446301
5     34750
1      6600
2      5105
3      4061
4      3183
Name: count, dtype: int64

▶ [인입횟수_ARS_B0M] 분포

In [11]:
# 검사할 컬럼 리스트
c1 = list (df.columns)
cols_to_check = c1

# 비수치형 컬럼 저장용 리스트
non_numeric_cols = []

# 각 컬럼에 대해 수치형 변환 시도 후 NaN 비율로 판단
for col in cols_to_check:
    # 결측을 빈 문자열로, 나머지를 문자열로 변환
    s = df[col].fillna('').astype(str)
    # 숫자로 변환 (불가능한 항목은 NaN)
    num = pd.to_numeric(s, errors='coerce')
    # 변환 불가 비율 계산
    na_ratio = num.isna().mean()
    # 일정 비율 이상이면 비수치형으로 간주 (여기선 100% 미수치형도 포함)
    if na_ratio > 0:
        non_numeric_cols.append(col)

# 결과 출력
print("비수치형(또는 혼합형) 컬럼:", non_numeric_cols)

비수치형(또는 혼합형) 컬럼: ['ID', '인입횟수_ARS_R6M', '이용메뉴건수_ARS_R6M', '방문횟수_PC_R6M', '방문일수_PC_R6M', '방문횟수_앱_R6M', 'OS구분코드', 'Segment']


In [12]:
# 숫자와 문자가 섞여 있는 컬럼 리스트 정의
cols = [
    '인입횟수_ARS_R6M',
    '이용메뉴건수_ARS_R6M',
    '방문횟수_PC_R6M',
    '방문일수_PC_R6M',
    '방문횟수_앱_R6M'
]

# 숫자 추출해서 새로운 수치형 컬럼 생성
for col in cols:
    # 문자열로 변환하고 숫자만 추출
    s = df[col].fillna('').astype(str)
    df[f'{col}_num'] = s.str.extract(r'(\d+)', expand=False)
    # 숫자로 변환, 변환 실패는 0으로 대체
    df[f'{col}_num'] = pd.to_numeric(df[f'{col}_num'], errors='coerce').fillna(0).astype(int)

# 원본 컬럼 삭제 
df.drop(columns=cols, inplace=True)

In [13]:
# 원본 결측 마스크 생성
mask_os_na = df['OS구분코드'].isna()

# 결측 여부 플래그 컬럼 추가
df['OS_missing_flag'] = mask_os_na.astype(int)

# OS구분코드 결측을 'Unknown'으로 대체
df['OS구분코드'] = df['OS구분코드'].fillna('Unknown')

# OS구분코드 원-핫 인코딩
dummies = pd.get_dummies(df['OS구분코드'], prefix='OS구분', dtype=int)

# 인코딩 결과를 df에 병합
df = pd.concat([df.drop(columns=['OS구분코드']), dummies], axis=1)

# 최종 df 컬럼 목록 출력 (검토용)
print(df.columns.tolist())
print(len(df.columns.tolist()))

['ID', '인입일수_ARS_R6M', '인입월수_ARS_R6M', '인입후경과월_ARS', '인입횟수_ARS_B0M', '이용메뉴건수_ARS_B0M', '인입일수_ARS_B0M', '방문월수_PC_R6M', '방문후경과월_PC_R6M', '방문일수_앱_R6M', '방문월수_앱_R6M', '방문후경과월_앱_R6M', '방문월수_모바일웹_R6M', '방문후경과월_모바일웹_R6M', '방문횟수_PC_B0M', '방문일수_PC_B0M', '방문횟수_앱_B0M', '방문일수_앱_B0M', '인입횟수_IB_R6M', '인입일수_IB_R6M', '인입월수_IB_R6M', '이용메뉴건수_IB_R6M', '인입후경과월_IB_R6M', '인입횟수_IB_B0M', '인입일수_IB_B0M', '이용메뉴건수_IB_B0M', '상담건수_B0M', 'IB문의건수_선결제_B0M', '상담건수_R6M', 'IB문의건수_선결제_R6M', '불만제기후경과월_R12M', '당사멤버쉽_방문횟수_B0M', '당사멤버쉽_방문횟수_R6M', '당사멤버쉽_방문월수_R6M', '홈페이지_금융건수_R6M', '홈페이지_선결제건수_R6M', '홈페이지_금융건수_R3M', '홈페이지_선결제건수_R3M', 'Segment', '인입횟수_ARS_R6M_num', '이용메뉴건수_ARS_R6M_num', '방문횟수_PC_R6M_num', '방문일수_PC_R6M_num', '방문횟수_앱_R6M_num', 'OS_missing_flag', 'OS구분_Android', 'OS구분_IOS', 'OS구분_Unknown']
48


In [14]:
# drop 이후, 저장 직전의 컬럼과 0/NaN 컬럼 재확인
zero_cols = [c for c in df.columns if df[c].eq(0).all()]
na_cols   = [c for c in df.columns if df[c].isna().any()]

print("저장 직전 0 전부 컬럼:", zero_cols)
print("저장 직전 NaN 있는 컬럼:", na_cols)
print("저장 직전 전체 컬럼 수:", len(df.columns))
print("저장 직전 전체 컬럼:", df.columns)

저장 직전 0 전부 컬럼: []
저장 직전 NaN 있는 컬럼: ['Segment']
저장 직전 전체 컬럼 수: 48
저장 직전 전체 컬럼: Index(['ID', '인입일수_ARS_R6M', '인입월수_ARS_R6M', '인입후경과월_ARS', '인입횟수_ARS_B0M',
       '이용메뉴건수_ARS_B0M', '인입일수_ARS_B0M', '방문월수_PC_R6M', '방문후경과월_PC_R6M',
       '방문일수_앱_R6M', '방문월수_앱_R6M', '방문후경과월_앱_R6M', '방문월수_모바일웹_R6M',
       '방문후경과월_모바일웹_R6M', '방문횟수_PC_B0M', '방문일수_PC_B0M', '방문횟수_앱_B0M',
       '방문일수_앱_B0M', '인입횟수_IB_R6M', '인입일수_IB_R6M', '인입월수_IB_R6M',
       '이용메뉴건수_IB_R6M', '인입후경과월_IB_R6M', '인입횟수_IB_B0M', '인입일수_IB_B0M',
       '이용메뉴건수_IB_B0M', '상담건수_B0M', 'IB문의건수_선결제_B0M', '상담건수_R6M',
       'IB문의건수_선결제_R6M', '불만제기후경과월_R12M', '당사멤버쉽_방문횟수_B0M', '당사멤버쉽_방문횟수_R6M',
       '당사멤버쉽_방문월수_R6M', '홈페이지_금융건수_R6M', '홈페이지_선결제건수_R6M', '홈페이지_금융건수_R3M',
       '홈페이지_선결제건수_R3M', 'Segment', '인입횟수_ARS_R6M_num', '이용메뉴건수_ARS_R6M_num',
       '방문횟수_PC_R6M_num', '방문일수_PC_R6M_num', '방문횟수_앱_R6M_num',
       'OS_missing_flag', 'OS구분_Android', 'OS구분_IOS', 'OS구분_Unknown'],
      dtype='object')


In [15]:
# csv 파일로 저장
df.to_csv('채널정보_전처리.csv', index=False)

In [16]:
# 저장된 파일 불러와서 확인하기
df2 = pd.read_csv('채널정보_전처리.csv')
zero_after = [c for c in df2.columns if df2[c].eq(0).all()]
na_after   = [c for c in df2.columns if df2[c].isna().any()]

print("로드 후 0 전부 컬럼:", zero_after)
print("로드 후 NaN 있는 컬럼:", na_after)
print("로드 후 전체 컬럼 수:", len(df2.columns))

로드 후 0 전부 컬럼: []
로드 후 NaN 있는 컬럼: ['Segment']
로드 후 전체 컬럼 수: 48
